## Load dataset

In [21]:
from sklearn.datasets import fetch_20newsgroups
from keras.utils import np_utils

cats = ['alt.atheism', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)
corpus = newsgroups_train.data
y_train = np_utils.to_categorical(newsgroups_train.target)

## Preprocessing

In [24]:
from zeugma import TextsToSequences, Padder
from sklearn.pipeline import make_pipeline

max_length = 200
num_words = 10000

sequencer = TextsToSequences(num_words=num_words)
padder = Padder(max_length=max_length)

# Build the Scikit-learn pipeline
pipeline = make_pipeline(sequencer, padder)
x_train = pipeline.fit_transform(corpus)

In [26]:
from zeugma import GloVeTransformer

embedding_transformer = GloVeTransformer(aggregation='average')
word_index = {k: v for k, v in sequencer.word_index.items() if v <= num_words}

def create_embedding_matrix(word_index, embedding_transformer):
    """ Prepare the embedding matrix """
    embedding_dim = embedding_transformer.transform(['the']).shape[1]
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
    for word, i in word_index.items():
        embedding_matrix[i] = embedding_transformer.transform([word])[0]
    return embedding_matrix

embedding_matrix = create_embedding_matrix(word_index, embedding_transformer)

LOG_DIR = os.path.join(os.environ['PWD'], 'logs')

if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
    
# Write word names in a file for tensorboard display
with open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w') as f:
    f.write('Word\n' + '\n'.join(word_index.keys()))

/Users/nicolas/Projects/zeugma/zeugma/embeddings.py:215: RuntimeWarning: invalid value encountered in true_divide
  text_vector = reduce(np.add, embeddings, np.zeros(size)) / len(tokens)


NameError: name 'os' is not defined

In [ ]:
embedding_layer_cvs = Embedding(len(embedding_matrix),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_cvs_len,
                            trainable=True,
                            name='embeddings_cvs')


job_input = Input(shape=(max_jobs_len,), dtype='int32')
job_embedded_sequences = embedding_layer_jobs(job_input)
#x_job = Flatten()(job_embedded_sequences)

conv_blocks = []
for filter_size in [3, 5, 7]:
    x = Conv1D(128, filter_size, activation='relu')(job_embedded_sequences)
    x = MaxPooling1D(filter_size)(x)
    x = Conv1D(128, filter_size, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    conv_blocks.append(x)
x_job = concatenate(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

In [ ]:
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, Input, Dropout
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(max_length, embedding_matrix):
    """ Model creation function: returns a compiled TextCNN"""
    embedding_layer = Embedding(len(embedding_matrix),
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=True,
                            name='embeddings')

    input_layer = Input(shape=(max_length,), dtype='int32', name='input')
    embedded_sequences = embedding_layer(input_layer)

    conv_blocks = []
    for filter_size in [3]:
        conv = Conv1D(128, filter_size, activation='relu', name='conv_'+str(filter_size))(embedded_sequences)
        pool = GlobalMaxPooling1D(name='global_pool_cv_'+str(filter_size))(conv)
        conv_blocks.append(pool)
    merge = concatenate(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    #dropout = Dropout(0.5)(merge)
    dense = Dense(128, activation='relu', name='dense')(merge)
    #dropout = Dropout(0.5)(dense)
    preds = Dense(20, activation='softmax', name='softmax')(dense)

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model

In [11]:
from sklearn.pipeline import make_pipeline

In [ ]:
make_pipeline(glove, )

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

